In [0]:
catalog_name = "dmc_tallerfinaldatabricks_mariovento"

schema_silver = "silver"
schema_gold = "gold"

In [0]:
from pyspark.sql.functions import to_date, col, year, month, dayofmonth, concat_ws, current_timestamp
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [0]:
ventas_silver = spark.table(f"{catalog_name}.{schema_silver}.ventas")

# Se crea una especificación de ventana para ordenar las fechas de forma ascendente y particionar por fecha
window_spec = Window.partitionBy("fecha").orderBy("fecha")

# Se construye la dimensión de tiempo a partir de las fechas de ventas
dim_tiempo = (
    ventas_silver
    .withColumn("fecha", to_date(col("fecha_venta")))  # Convierte la columna fecha_venta a tipo fecha
    .filter(col("fecha").isNotNull())  # Elimina valores null de fecha
    .select("fecha")  # Selecciona solo la columna fecha
    .dropDuplicates(["fecha"])  # Elimina duplicados de fechas
    .withColumn("anio", year(col("fecha")).cast("int"))
    .withColumn("mes", month(col("fecha")).cast("int"))
    .withColumn("dia", dayofmonth(col("fecha")).cast("int"))
    .withColumn("anio_mes", concat_ws("-", col("anio"), col("mes")).cast("string"))
    # Genera una clave secuencial para cada fecha y la castea a BIGINT
    .withColumn("sk_tiempo", row_number().over(window_spec).cast("bigint"))
    .withColumn("creation_date", current_timestamp())  # Fecha de creación del registro
    .select(
        col("sk_tiempo"),
        col("fecha"),
        col("anio"),
        col("mes"),
        col("dia"),
        col("anio_mes"),
        col("creation_date")
    )
)

# Particiona la tabla por año y mes para mejorar el rendimiento en consultas
(
    dim_tiempo
    .write
    .format("delta")
    .mode("overwrite")
    .partitionBy("anio", "mes")
)

In [0]:
dim_tiempo.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_gold}.dim_tiempo")

In [0]:
%sql
select * from dmc_tallerfinaldatabricks_mariovento.gold.dim_tiempo